In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import random

In [3]:
# Veri Yükleme ve Temizleme
df = pd.read_csv("civic-yeni-data.csv")

data = df.copy()
data['price'] = data['price'].str.replace('.', '', regex=False)
data['price'] = data['price'].str.replace(' TL', '', regex=False)
data['price'] = pd.to_numeric(data['price'], errors='coerce')
#fiyattaki tl ve noktaları siler
def extract_number(value):
    if pd.isna(value):
        return np.nan
    num = ''.join(filter(str.isdigit, value.split()[0]))
    return int(num) if num else np.nan

data['enginePower'] = data['enginePower'].apply(extract_number)
data['engineCapacity'] = data['engineCapacity'].apply(extract_number)
 #125 HP” gibi string’lerden ilk sayıyı çekip int’e dönüştürür.
data.dropna(subset=['year', 'mileage', 'price', 'enginePower', 'engineCapacity'], inplace=True)
#boş olan satırları atma
categorical_cols = ['location', 'transmission', 'color', 'model', 'fuelType']
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
#kategorik sutunlar one hot a çevirilir.
drop_cols = ['title', 'painted', 'changed','paintedCount']
data.drop(columns=drop_cols, inplace=True)
#istenmeyen sutunlerin atılması

In [4]:
# 3. Değişken Ayırma
X = data.drop(columns=["price"])
y = data["price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
#Genetik Algoritma ile Özellik Seçimi
population_size = 30
generations = 50
mutation_rate = 0.05
#population_size: Her nesilde 30 birey (özellik kombinasyonu) tutulur.
#generations: 50 nesil boyunca evrim yapılır.
#mutation_rate: %5 ihtimalle her genin (özellik 0/1 durumu) mutasyona uğrama olasılığı.
def random_individual(n_features):
    # Manuel olarak belirlediğin özellik seçimi
    return [0,1, 1, 1, 0, 0, 1, 0, 1, 1, 1,0,0,0,0]
#
#
def fitness(individual, X_train, y_train, X_val, y_val):
    selected_features = [i for i, bit in enumerate(individual) if bit == 1]
    if not selected_features:
        return float('-inf')
    model = LinearRegression()
    model.fit(X_train.iloc[:, selected_features], y_train)
    preds = model.predict(X_val.iloc[:, selected_features])
    mse = mean_squared_error(y_val, preds)
    return -mse
#individual: Bir birey = [1, 0, 0, 1, ...] gibi bir dizi → 1 olanlar seçili özellik
#selected_features: Sadece 1 olan indexler tutulur (örn. [0, 3, 4, 7])
#Model kurulup eğitilir → X_train ve X_val içinden sadece bu sütunlar alınır.
#mean_squared_error hesaplanır
#Amaç: Hata küçük olsun → bu yüzden -mse döndürülür (büyük fitness = iyi birey)
#
def evolve(population, X_train, y_train, X_val, y_val):
    sorted_population = sorted(population, key=lambda ind: fitness(ind, X_train, y_train, X_val, y_val), reverse=True)
    next_generation = sorted_population[:2]

    while len(next_generation) < population_size:
    #En iyi 5 birey arasından rastgele 2 ebeveyn seçilir (seleksiyon baskısı).
        parent1, parent2 = random.choices(sorted_population[:5], k=2)
        crossover_point = random.randint(1, len(parent1) - 2)
        child = parent1[:crossover_point] + parent2[crossover_point:] #Tek noktalı çaprazlama
        child = [bit if random.random() > mutation_rate else 1 - bit for bit in child] # mutasyon yapılması
        next_generation.append(child) # yeni birey eklenir
    return next_generation
#Bireyler sıralanır (en yüksek fitness’lılar en başa).
#Elitizm uygulanır → en iyi 2 birey doğrudan yeni nesile geçer.

X_subtrain, X_val, y_subtrain, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=1)
population = [random_individual(X.shape[1]) for _ in range(population_size)]
#Eğitim verisi, alt eğitim (%70) ve validasyon (%30) olarak bölünür
#Başlangıç popülasyonu oluşturulur
#50 nesil boyunca evrim uygulanır

for gen in range(generations):
    population = evolve(population, X_subtrain, y_subtrain, X_val, y_val)
best_individual = max(population, key=lambda ind: fitness(ind, X_subtrain, y_subtrain, X_val, y_val))
selected_features = [i for i, bit in enumerate(best_individual) if bit == 1]
feature_cols = X_train.columns[selected_features].tolist()
#Son nesildeki en iyi birey seçilir
#1 olan özellikler alınır (seçilen feature indexleri)
#Bu indexler üzerinden isim listesi (feature_cols) çıkarılır → model eğitimde burası kullanılır



In [6]:
#Model Eğitimi ve Test
X_train_selected = X_train.iloc[:, selected_features]
X_test_selected = X_test.iloc[:, selected_features]
#
#
final_model = LinearRegression(positive=True)
final_model.fit(X_train_selected, y_train)
#
#
y_pred = final_model.predict(X_test_selected)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
#
#
print("Model RMSE:", rmse)


Model RMSE: 11119132.300178856


In [8]:
# Model Test Etme
def prepare_user_input(raw_dict):
    df_u = pd.DataFrame([raw_dict])

    # dummy price (zorunlu değil ama sütun sayısı eşit kalsın diye)
    df_u['price'] = 0

    # sayısal kolonları güvenle çevir
    for c in ['year', 'mileage', 'enginePower', 'engineCapacity']:
        df_u[c] = pd.to_numeric(df_u[c], errors='coerce')
    # one-hot kodla
    df_u = pd.get_dummies(df_u, columns=categorical_cols, drop_first=True)
    #eksik olanları 0 ile doldur
    #fazla olanları at
    df_u = df_u.reindex(columns=X.columns, fill_value=0)

    # GA’nin seçtiği sütunları çek
    return df_u[feature_cols]

# Kullanıcıdan Giriş Al
print("\n--- Aracınızın bilgilerini girin ---")
u_in = {
    "year"          : input("Model Yılı         : "),
    "mileage"       : input("Kilometre (km)     : "),
    "enginePower"   : input("Motor Gücü (HP)    : "),
    "engineCapacity": input("Motor Hacmi (cc)   : "),
    "location"      : "Ankara, Çankaya",
    "transmission"  : input("Şanzıman Tipi      : "),
    "color"         : input("Renk               : "),
    "model"         : input("Donanım Paketi     : "),
    "fuelType"      : input("Yakıt Türü         : "),

}

try:
    X_user = prepare_user_input(u_in)
    price_pred = final_model.predict(X_user)[0]
    print(f"\n→ Tahmini Fiyat: {int(price_pred):,} TL")
except Exception as e:
    print("⚠️ Girilen veriler işlenirken hata:", e)



--- Aracınızın bilgilerini girin ---

→ Tahmini Fiyat: 1,129,101 TL
